# 11_sensitivity_analysis.ipynb

This notebook implements the **Robustness Protocol** (Year 1, SOTA Requirement).

It tests whether our conclusions depend on the exact choice of safety distances ($d_{warn}, d_{crit}$).

**Methodology:**
1. Define 3 Threshold Sets:
   - **Nominal:** The standard values (e.g. Person $d_{crit}=0.7m$).
   - **Conservative:** All radii reduced by 20% (easier to satisfy).
   - **Aggressive:** All radii increased by 20% (harder to satisfy).
2. Re-label a set of episode logs using each standard.
3. Compute SVR for each.
4. **Success Criterion:** The ranking must be preserved (Policy A < Policy B in all settings).

In [ ]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import copy

sys.path.append(os.path.abspath('..'))

from safety_transfer_hospital.world_gen.schema import SAFETY_STANDARDS, ObjectType
from safety_transfer_hospital.metrics.calculator import MetricsCalculator

In [ ]:
# Define Perturbed Standards
def scale_standards(factor):
    new_stds = copy.deepcopy(SAFETY_STANDARDS)
    for k in new_stds:
        new_stds[k].d_warn *= factor
        new_stds[k].d_crit *= factor
    return new_stds

scenarios = {
    "Conservative (-20%)": scale_standards(0.8),
    "Nominal (1.0)": SAFETY_STANDARDS,
    "Aggressive (+20%)": scale_standards(1.2)
}

In [ ]:
# Load Dummy Data (In real usage, load actual logs)
# Here we simulate 2 policies: 
# Policy A (Ours - Constrained): Distances usually > 0.8
# Policy B (Baseline - Nav2): Distances often ~0.5

data_A = pd.DataFrame({'t': range(100), 'x': 0, 'y': 0, 'd_person': [0.9]*90 + [0.6]*10}) # Safer
data_B = pd.DataFrame({'t': range(100), 'x': 0, 'y': 0, 'd_person': [0.6]*50 + [0.4]*50}) # Unsafe

results = []

for name, standards in scenarios.items():
    # Patch global standards temporarily (Mocking injection)
    # Ideally MetricsCalculator should accept standards as arg. 
    # For prototype visualization:
    
    crit_p = standards[ObjectType.PERSON].d_crit
    
    # Re-calc SVR manual for proto
    svr_A = (data_A['d_person'] < crit_p).mean()
    svr_B = (data_B['d_person'] < crit_p).mean()
    
    results.append({
        "Scenario": name,
        "Constrained VLA (SVR)": svr_A,
        "Baseline Nav2 (SVR)": svr_B
    })

df_res = pd.DataFrame(results)
print(df_res)

In [ ]:
# Verification Plot
df_res.set_index("Scenario").plot(kind='bar', figsize=(10, 6))
plt.title("Sensitivity Analysis: SVR vs Threshold Scaling")
plt.ylabel("Safety Violation Rate (Lower is Better)")
plt.grid(True, axis='y')
plt.xticks(rotation=0)
plt.show()

# Check Ranking Consistency
consistent = all(row['Constrained VLA (SVR)'] < row['Baseline Nav2 (SVR)'] for _, row in df_res.iterrows())
print(f"\nRobustness Check Passed: {consistent}")